<a href="https://colab.research.google.com/github/miaanastasia/lstm_pipeline/blob/main/LSTM_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LSTM-Based Stock Price Prediction**

This notebook demonstrates a stock prediction pipeline using machine learning and LSTM neural networks, with technical and market-based features, visual evaluation, and an interactive UI for exploring results.

> *This program is optimized for Google Colab Pro accounts with GPU access and at least ten available computing units.*

## User Guide

#### Processor
1. Click `Connect` at the top right of the notebook.
2. Select `L4 GPU` or `T4 GPU` from the runtime options.
3. Verify connectivity with green check mark.

#### Pipeline
1. In the Colab menu bar, go to **Runtime > Run All.**
2. The default ticker is set to `DOW`.
3. Wait a few moments while all cells execute.
4. Once complete, check the task bar at the bottom for confirmation.


> ***Tips:***
> - Collapse sections using the bold headers for cleaner navigation.
> - To run a specific section, hover over the `[ ]` icon below the collapsed header and click the play button.

#### User Interface
1. Scroll to the **User Interface** section at the bottom of the notebook.
2. The dashboard appears by default under the `Pipeline` tab.
3. Select a ticker from the dropdown and click 'Run Pipeline'.
4. Watch the model train and view output loss and average percentage error.
5. Navigate to the `Visualization` tab explore various charts and performance metrics.
6.Select a visualization and click 'Show' (some may take a few moments to load).
7. Return to the `Pipeline` tab to reset the model and try a new ticker.



## **Languages, Packages, & Libraries**

### Python

In [ ]:
# !sudo update-alternatives --config python3
!sudo apt-get install python3.10

In [ ]:
!python --version

### Widgets

In [ ]:
!pip install ipywidgets==8.1.1 --quiet

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

### Imports

#### Data Exploration

In [ ]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from pandas_datareader import data as pdr
import seaborn as sns
import os

#### Machine Learning

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error
from sklearn.inspection import permutation_importance

import tensorflow as tf
import keras
from keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

In [ ]:
print('Tensorflow', tf.__version__)
print('Keras', keras.__version__)

### Seeds

In [ ]:
# set seeds for reproducibility
def set_seed(seed=42):
  np.random.seed(seed)
  tf.random.set_seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)

## **Exploratory Data Analysis**

### Global Definitions

In [ ]:
# default stock or index ticker
ticker = 'DOW'

# look-back window (days)
lookback = 60

# train and test split
split_ratio = 0.8

### Create DataFrame

In [ ]:
def load_data(ticker, start='2019-03-01', end='2025-03-01'):

  """
  Downloads stock or market index data and merges with closing values for
  Nasdaq and S&P 500.

  Parameters:
  - ticker: Ticker symbol chosen by user, default 'DOW'
  - start: Starting date of data collection
  - end: Ending date of data collection

  Returns:
  - df: Merged DataFrame
  """

  # download individual stock data
  df_stock = yf.download(ticker, start=start, end=end, progress=False)

  # download market index data
  indexes = ['NQ=F', 'SPGI']
  df_indexes = {}

  for index in indexes:
    df_index = yf.download(index, start=start, end=end, progress=False)
    # ensure df_indexes only has one level before merging
    df_indexes[index] = df_index['Close'].squeeze()

  # flatten the multiIndex (removes 'Ticker' level)
  df_stock = df_stock.T.droplevel(1).T

  # convert dictionary to dataframe before merging
  df_indexes = pd.DataFrame(df_indexes)
  df_indexes.rename(columns={'NQ=F': 'NQ_Close', 'SPGI': 'SPGI_Close'}, inplace=True)

  df = df_stock.merge(df_indexes, on='Date', how='left')

  return df

load_data(ticker)

## **Feature Engineering**

### Default Features

In [ ]:
def build_default_features(df):

  """
  Adds technical indicators and features based on the individual stock.

  Parameters:
  - df: Merged DataFrame of stock/market data

  Returns:
  - df: Merged DataFrame with individual stock features
  """

  # ----------------------------VOLUME------------------------------------
  # moving averages: smooth out fluctuations and identify trends
  df['MA_10'] = df['Close'].rolling(window=10).mean()
  df['MA_50'] = df['Close'].rolling(window=50).mean()
  df['MA_200'] = df['Close'].rolling(window=200).mean()

  # on-balance volume: identifies buying and selling pressure
  df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()

  # volume-weighted avg price: avg price a stock traded at throughout the day
  df['VWAP'] = (df['Close'] * df['Volume']).cumsum() / df['Volume'].cumsum()


  # ----------------------------VOLATILITY---------------------------------
  # rolling standard deviation: measures fluctuations over a given window
  df['Rolling_std_10'] = df['Close'].rolling(window=10).std()
  df['Rolling_std_20'] = df['Close'].rolling(window=20).std()

  # Bollinger Bands: identify overbought/oversold conditions
  df['BB_Middle'] = df['Close'].rolling(window=20).mean() # 20-day moving avg
  df['BB_Upper'] = df['BB_Middle'] + (df['Rolling_std_20'] * 2)
  df['BB_Lower'] = df['BB_Middle'] - (df['Rolling_std_20'] * 2)


  # ------------------------------MOMENTUM---------------------------------
  # rate of change: how much the price has changed over n days
  df['ROC_10'] = df['Close'].pct_change(periods=10) * 100
  df['ROC_20'] = df['Close'].pct_change(periods=20) * 100

  # relative strength index
  # df['RSI'] = df['Close']

  # williams %R: used with BB to identify overbought/oversold conditions
  df['WilliamsR'] = (df['High'].rolling(14).max() - df['Close']) / (
      df['High'].rolling(14).max() - df['Low'].rolling(14).min()) * -100

  df.dropna(inplace=True)

  return df

### Market Index Features (Nasdaq, S&P Global)

In [ ]:
def build_market_features(df):

  """
  Adds market-based indicators and correlation with the chosen stock.

  Parameters:
  - df: Merged DataFrame

  Returns:
  - df: Merged DataFrame with stock and market index features
  """

  try:
    # compute rolling correlation between stock, sp500, NASDAQ
    df['Corr_NQ'] = df['Close'].rolling(50).corr(df['NQ_Close'])
    df['Corr_SPGI'] = df['Close'].rolling(50).corr(df['SPGI_Close'])

    # moving averages of market indices
    df['NQ_MA_50'] = df['NQ_Close'].rolling(window=50).mean()
    df['SPGI_MA_50'] = df['SPGI_Close'].rolling(window=50).mean()

    # market rate of change
    df['NQ_ROC_10'] = df['NQ_Close'].pct_change(periods=10) * 100
    df['SPGI_ROC_10'] = df['SPGI_Close'].pct_change(periods=10) * 100

    df.dropna(inplace=True)

    # print('[DEBUG] Market features added.  Shape:', df.shape)

    return df
  except Exception as e:
    print('[ERROR] Failed to add market features:', e)
    return None

### Global Definition of Full Feature Set

In [ ]:
FEATURES = ['Close', 'NQ_Close', 'SPGI_Close', 'MA_10', 'MA_50', 'MA_200',
              'OBV', 'VWAP', 'Rolling_std_10', 'Rolling_std_20', 'BB_Middle',
              'BB_Upper', 'BB_Lower', 'ROC_10', 'ROC_20', 'WilliamsR',
              'Corr_NQ', 'Corr_SPGI', 'NQ_MA_50', 'SPGI_MA_50']

## **Data Preprocessing**

### Prepare Data

In [ ]:
def prepare_data(ticker, start='2015-01-01', end='2025-01-01'):

  """
  Loads stock and market index data for a given ticker and adds all features.

  Parameters:
  - ticker: Ticker symbol set by user, default 'AAPL'
  - start: Starting date of data collection
  - end: Ending date of data collection

  Returns:
  - df: Fully processed DataFrame
  """

  df = load_data(ticker, start=start, end=end)
  df = build_default_features(df)
  df = build_market_features(df)

  print('\nStock and market data downloaded, DataFrame created.\nIndividual stock features calculated.\nMarket index features calculated.')

  # clean extraneous columns
  df = df.drop(['High', 'Low', 'Open', 'Volume'], axis=1)

  print('\nDataFrame ready for preprocessing.\n')

  return df

df = prepare_data(ticker)

### Preprocess Data

In [ ]:
def preprocess_data(df, lookback, split_ratio, features=FEATURES):

  """
  Prepares the data for training by scaling relevant features down to values
  between 0 and 1 for interpretability.  Splits the data into train and test
  sets.

  Parameters:
  - df: Merged dataframe of stock/market data
  - lookback: Globally-defined lookback window
  - split_ratio: Globally defined split ratio for train and test sets
  - features: Globally-defined feature set

  Returns:
  X_train, X_test, y_train, y_test, scaler, df_features
  """

  scaler = MinMaxScaler()
  df_features = df[features]

  scaled_data = scaler.fit_transform(df_features)

  X, y = [], []

  for i in range(lookback, len(scaled_data)):
    X.append(scaled_data[i - lookback:i])
    y.append(scaled_data[i, 0]) # only predicting 'Close' price

  # cast X and y sequence lists to arrays
  X, y = np.array(X), np.array(y)
  y = y.reshape(-1) # flatten to 1-D array

  # split data into train and test sets along time axis
  split_index = int(len(X)*split_ratio)
  X_train, X_test = X[:split_index], X[split_index:]
  y_train, y_test = y[:split_index], y[split_index:]

  print('DataFrame preprocessed and ready for modeling.\n\n')

  return X_train, X_test, y_train, y_test, scaler, df_features

X_train, X_test, y_train, y_test, scaler, df = preprocess_data(df, lookback, split_ratio)

### Correlation Matrix of Features

In [ ]:
def show_feature_correlation(df):

  """
  Plots a correlation matrix of the features after scaling.

  Parameters:
  - df: DataFrame of features from preprocess_data()
  """

  corr_matrix = df.corr()
  plt.figure(figsize=(10,8))
  sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
  plt.title('Feature Correlation Matrix')
  plt.show()

show_feature_correlation(df)

## **Modeling**

**Recall:**

X_train.shape[0] --> N --> samples

X_train.shape[1] --> T --> time

X_train.shape[2] --> D --> features

### Model Configuration

In [ ]:
def get_model_config(ticker):
  """
  Returns model hyperparameters based on whether the ticker is an index or
  individual stock.
  """

  stock_tickers = ['AAPL', 'FOX', 'JPM', 'NFLX', 'SPOT']
  index_tickers = ['DOW', '^GSPC', '^IXIC', '^RUT']

  # individual stock hyperparameter optimization
  if ticker in stock_tickers:
    config = {
        # 'dense_unit_1': 25,
        # 'dropout_1': 0.2,
        # 'dense_unit_2': 25,
        # 'dropout_2': 0.3

        'dense_unit_1': 25,
        'dropout_1': 0.2,
        'dense_unit_2': 50,
        'dropout_2': 0.3
    }
  elif ticker in index_tickers:
    config = {
        'dense_unit_1': 25,
        'dropout_1': 0.1,
        'dense_unit_2': 50,
        'dropout_2': 0.1
    }
  else:
    config = {'dropout_1': 0.2, 'dropout_2': 0.2,
              'dense_unit_1': 25, 'dense_unit_2': 50}

  print(f"Training model for {ticker} with configuration: {config}\n\n")
  return config

config = get_model_config(ticker)

### Model Architecture

In [ ]:
def train_lstm(X_train, y_train, X_test, y_test, epochs=75, batch_size=32,
                                                              config=config):

  """
  Trains and compiles a sequential model on an LSTM neural network with
  time series data.

  Parameters:
  - X_train: Training set input features
  - y_train: Training set actual values
  - X_test: Test set input features
  - y_test: Test set actual values
  - epochs: Number of full iterations
  - batch_size: Number of training samples processed in a single epoch

  Returns:
  model, history
  """

  input_shape = (X_train.shape[1], X_train.shape[2])

  model = Sequential([
        Input(shape=input_shape),
        LSTM(100, return_sequences=True),
        Dense(config['dense_unit_1']),
        Dropout(config['dropout_1']),
        LSTM(50),
        Dense(config['dense_unit_2']),
        Dropout(config['dropout_2']),
        Dense(1)
    ])

  model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')
  history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                                    epochs=epochs, batch_size=batch_size,
                                                              verbose=2)

  return model, history

### Train and Compile

In [ ]:
lstm_model, history = train_lstm(X_train, y_train, X_test, y_test)
lstm_predictions = lstm_model.predict(X_test)

### Model Summary

In [ ]:
def print_model_summary(model):
  model.summary()

summary = print_model_summary(lstm_model)

## **Validation**

### Calculate Loss (MSE, RMSE, MAE)

In [ ]:
def calculate_loss(y_test, lstm_predictions):

  mse = mean_squared_error(y_test, lstm_predictions)
  rmse = root_mean_squared_error(y_test, lstm_predictions)
  mae = mean_absolute_error(y_test, lstm_predictions)

  print(f'\nMSE: {mse}')
  print(f'RMSE: {rmse}')
  print(f'MAE: {mae}\n')

calculate_loss(y_test, lstm_predictions)

### Plot Validation Loss (MSE)

In [ ]:
def plot_training_history(history):

  """
  Plots the training and validation loss for each epoch.

  Parameters:
  - history: fitted model
  """

  plt.figure(figsize=(10,5))
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')

  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.title('Training vs Validation Loss')
  plt.legend()
  plt.show()

plot_training_history(history)

## **Evaluation**

### Feature Importance -- Permutation Method

In [ ]:
def permutation_feature_importance(model, X_test, y_test, features=FEATURES):

  """
  Evaluates feature importance by measuring the increase in MSE after shuffling
  each feature in X_test.

  Parameters:
  - model: Trained LSTM model
  - X_test: Test set input features
  - y_test: Test set actual values
  - features: Global feature set

  Returns:
  df of features sorted by importance
  """

  baseline_pred = model.predict(X_test, verbose=0)
  baseline_mse = mean_squared_error(y_test, baseline_pred)

  importances = {}

  for i, feature in enumerate(features):
    X_permuted = np.copy(X_test)

    # shuffle values for this feature across all samples and timesteps
    for t in range(X_permuted.shape[1]):
      np.random.shuffle(X_permuted[:, t, i])

    permuted_pred = model.predict(X_permuted, verbose=0)
    permuted_mse = mean_squared_error(y_test, permuted_pred)

    # importance is the increase in error caused by shuffling this feature
    importances[feature] = permuted_mse - baseline_mse

  # sort and display results
  importance_df = pd.DataFrame(importances.items(), columns=['Feature', 'Importance']
                               ).sort_values(by='Importance', ascending=False)

  # print("\nFeature Importance (Permutation Method):")
  # print(importance_df)

  return importance_df

importance_df = permutation_feature_importance(lstm_model, X_test, y_test, FEATURES)
print(importance_df)

### Plot Top Feature Importances

In [ ]:
def plot_feature_importance(importance_df, top_n=10):

  plt.figure(figsize=(10,6))
  sns.barplot(data=importance_df.head(top_n), x='Importance', y='Feature', hue='Feature',
              palette='crest', legend=False)
  plt.title('Top Feature Importances (Permutation Method)')
  plt.xlabel('MSE Increase on Feature Shuffle')
  plt.ylabel('Feature')
  plt.grid(True, axis='x')
  plt.tight_layout()
  plt.show();

plot_feature_importance(importance_df)

### Inverse Transform Predictions

In [ ]:
def inverse_transform(y_test, predictions, scaler, features=FEATURES):

  """
  Inverses the scaling of predictions back to original prices.

  Parameters:
  - y_test: Ground truth values (scaled)
  - predictions: Predicted values from model (scaled)
  - scaler: the fitted MinMaxScaler used on the feature set
  - features: list of features used during scaling

  Returns:
  true_close, adj_close
  """

  # modify scaler.inverse_transform() to include features
  y_test_rescaled = np.zeros((y_test.shape[0], len(features)))
  y_test_rescaled[:,0] = y_test

  # modify scaler.inverse_transform() to include features
  preds_rescaled = np.zeros((predictions.shape[0], len(features)))
  preds_rescaled[:,0] = predictions[:,0]

  # extract the first column (closing price)
  adj_close = scaler.inverse_transform(preds_rescaled)[:,0]
  true_close = scaler.inverse_transform(y_test_rescaled)[:,0]

  return true_close, adj_close

true_prices, predicted_prices = inverse_transform(y_test, lstm_predictions, scaler)

### Calculate Error

In [ ]:
def compute_error(true, adj):
  pct_error = np.mean(np.abs(true-adj) / true) * 100
  print(f'\nAverage Percentage Error: {pct_error:.2f}%\n')

  return pct_error

error = compute_error(true_prices, predicted_prices)

### Plot True vs Predicted Prices

In [ ]:
def plot_predictions_with_dates(df, true_prices, predicted_prices, lookback,
                                                              split_ratio):

  """
  Plots actual vs. predicted stock prices over time with proper date labels.

  Parameters:
  - df: Original DataFrame with full date index
  - true_prices: Actual prices after inverse transform
  - predicted_prices: Model predictions after inverse transform
  - lookback: number of days in input window
  - split_ratio: train/test split ratio
  """

  # align with end of test set
  total_samples = lookback + len(true_prices)
  dates = df.index[-len(true_prices):]

  plt.figure(figsize=(12,6))
  plt.plot(dates, true_prices, label='True Prices')
  plt.plot(dates, predicted_prices, label='Predicted Prices')
  plt.xlabel('Date')
  plt.ylabel('Price')
  plt.title('LSTM Stock Price Predictions')
  plt.legend()
  plt.grid(True)
  plt.tight_layout()
  plt.xticks(rotation=45)
  plt.show()

plot_predictions_with_dates(df, true_prices, predicted_prices, lookback, split_ratio)

### DataFrame of Closing Prices

In [ ]:
def compare_prices(df, y_test, true_prices, predicted_prices):

  df_close = pd.DataFrame({
      'Date': df.index[-len(y_test):],
      'True Close': true_prices,
      'Adjusted Close': predicted_prices,
  })

  print('\nDataFrame of True vs Predicted Prices\n')
  print(df_close)

compare_prices(df, y_test, true_prices, predicted_prices)

## **Pipeline Script**

In [ ]:
def run_prediction_pipeline(ticker):
    """
    Loads and preprocesses data, trains the model, and generates predictions.
    Saves results to the global `state` dictionary and key variables to the global namespace.
    Returns the full `state` for optional inspection.
    """

    global df, history, X_test, y_test, true_prices, predicted_prices, model, error

    print(f"\nRunning LSTM prediction for {ticker}...\n")

    # Load and preprocess
    df = prepare_data(ticker)
    X_train, X_test, y_train, y_test, scaler, df = preprocess_data(df, lookback, split_ratio)

    # Train model
    model_config = get_model_config(ticker)
    model, history = train_lstm(X_train, y_train, X_test, y_test, config=model_config)

    # Predict
    predictions = model.predict(X_test)

    # Validate
    loss = calculate_loss(y_test, predictions)

    # Evaluate
    true_prices, predicted_prices = inverse_transform(y_test, predictions, scaler)
    error = compute_error(true_prices, predicted_prices)

    return df, history, X_test, y_test, true_prices, predicted_prices, error

## **User Interface**

In [ ]:
state = {
    'df': None,
    'X_test': None,
    'y_test': None,
    'true_prices': None,
    'predicted_prices': None,
    'model': None,
    'history': None,
    'error': None,
    'feature_list': None
}

import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import VBox, HBox, Tab
import io
from contextlib import redirect_stdout

# --- Global State ---
pipeline_output = widgets.Output()
viz_output = widgets.Output()
# df = model = history = true_prices = predicted_prices = y_test = X_test = None

# --- Widgets ---
ticker_dropdown = widgets.Dropdown(
    options=['AAPL', 'AXP', 'DOW', 'FOX', 'GOOGL', '^GSPC', '^IXIC', 'JPM', 'LMT', 'MA', 'MSFT',
             'NFLX', '^RUT', 'SPOT'],
    value='DOW',
    description='Ticker:',
    layout=widgets.Layout(width='200px')
)

run_button = widgets.Button(
    description='Run Pipeline',
    button_style='success',
    layout=widgets.Layout(width='150px')
)

viz_dropdown = widgets.Dropdown(
    options=[
        'Feature Correlation Matrix',
        'Model Summary',
        'Training vs Validation Loss',
        'Feature Importance',
        'True vs Predicted Prices',
        'Price Comparison Table'
    ],
    description='Visual:',
    layout=widgets.Layout(width='350px')
)

show_button = widgets.Button(
    description='Show',
    button_style='info',
    layout=widgets.Layout(width='100px')
)

reset_button = widgets.Button(
    description='Reset All',
    button_style='danger',
    layout=widgets.Layout(width='120px')
)

# --- Utility: Get Model Summary as Text (avoid infinite loop) ---
def get_model_summary_text(model):
    buf = io.StringIO()
    with redirect_stdout(buf):
        model.summary()
    return buf.getvalue()

# --- Callbacks ---
def on_run_click(b):
    with pipeline_output:
        clear_output(wait=True)
        ticker = ticker_dropdown.value

        df, history, X_test, y_test, true_prices, predicted_prices, error = run_prediction_pipeline(ticker)

        state.update({
            'df': df,
            'history': history,
            'y_test': y_test,
            'true_prices': true_prices,
            'predicted_prices': predicted_prices,
            'error': error,
            'model': model,
            'X_test': X_test,
            'feature_list': FEATURES
        })

        print(f"Model trained for {ticker} with {len(df)} data points.")


def on_show_click(b):

    with viz_output:
        clear_output(wait=True)
        if state['model'] is None:
            print("⚠️ Please run the pipeline first.")
            return

        choice = viz_dropdown.value
        if choice == 'Model Summary':
            print(get_model_summary_text(state['model']))
        elif choice == 'True vs Predicted Prices':
            plot_predictions_with_dates(state['df'], state['true_prices'], state['predicted_prices'], lookback, split_ratio)
        elif choice == 'Feature Importance':
            importance_df = permutation_feature_importance(state['model'], state['X_test'], state['y_test'], state['feature_list'])
            plot_feature_importance(importance_df)
        elif choice == 'Training vs Validation Loss':
            plot_training_history(state['history'])
        elif choice == 'Feature Correlation Matrix':
            show_feature_correlation(state['df'])
        elif choice == 'Price Comparison Table':
            compare_prices(state['df'], state['y_test'], state['true_prices'], state['predicted_prices'])
        else:
            print("Select a visualization.")


def on_reset_click(b):
    for key in state:
        state[key] = None
    with pipeline_output:
        clear_output()
        print("Pipeline output cleared.")
    with viz_output:
        clear_output()
        print("Visualization output cleared.")

# --- Connect Buttons ---
run_button.on_click(on_run_click)
show_button.on_click(on_show_click)
reset_button.on_click(on_reset_click)

# --- Layout ---
pipeline_tab = VBox([
    HBox([ticker_dropdown, run_button, reset_button]),
    pipeline_output
])

visual_tab = VBox([
    HBox([viz_dropdown, show_button]),
    viz_output
])

tabs = Tab(children=[pipeline_tab, visual_tab])
tabs.set_title(0, 'Pipeline')
tabs.set_title(1, 'Visualization')

# --- Display UI ---
display(tabs)
